In [160]:
class ManagedFile():
    def __init__(self, name):
        self.name = name
        
    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()
            
with ManagedFile('simple1.csv') as f:
    f.write('"Hello,",')
    f.write('"world!"')

In [161]:
class Indenter():
    def __init__(self, indent=0):
        self.indent = indent
    
    def __enter__(self):
        self.indent += 1
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.indent -= 1
        
    def print(self, text):
        self.text = text
        print('    '*self.indent + self.text)



In [ ]:
with Indenter() as indent:
    print('hello!')
    indent.print('hi')
    with indent:
        indent.print('hello!')
        with indent:
            indent.print('indented even further')

In [ ]:
from contextlib import contextmanager

@contextmanager
def managed_file(name):
    try:
        f = open(name, 'w')
        yield f
    finally:
        f.close()

with managed_file('hello.txt') as f:
    f.write('hello, world!')
    f.write('bye now')

In [162]:
class contextmanager:
    def __init__(self, gen):
        self.gen = gen
    def __call__(self, *args, **kwargs):
        self.args, self.kwargs = args, kwargs
        return self
    def __enter__(self):
        self.gen_inst = self.gen(*self.args, **self.kwargs)
        next(self.gen_inst)
    def __exit__(self, *args):
        next(self.gen_inst, None)
        

In [163]:
from contextlib import contextmanager

@contextmanager
def indenter(level=0):
    def prints(text):
        print('____' * level + text)
    try:
        level += 1
        yield prints
    finally:
        level -= 1
    
            
with indenter() as ind:
    print('0000')
    ind('1111')
    with indenter(1) as ind:
        ind('2222')
    with indenter(2) as ind:
        ind('3333')

#wrong!


0000
____1111
________2222
____________3333


In [169]:
from contextlib import contextmanager

def indenter():
    level = 0

    def prints(text):
        print('____' * level + text)

    @contextmanager
    def switcher():
        nonlocal level
        try:
            level += 1
            yield prints
        finally:
            level -= 1
    return switcher


ind = indenter()
with ind() as print_ind:
    print('aaa')
    print_ind('aaa')
    with ind() as print_ind:
        print_ind('bbb')
        with ind() as print_ind:
            print_ind('ccc')

aaa
____aaa
________bbb
____________ccc


In [22]:
#closures are functions with preserved data
def foo(num):
    def bar(text):
        print(text, num)
    return bar

s = foo(42)
s('The answer is')

The answer is 42


In [33]:
ss = [1,2,3,4]

In [42]:
s = ', '.join([str(i) for i in ss])

In [43]:
s

'1, 2, 3, 4'

# Lambdas
Basically labdas are nameless functions, they work as a functions:
`lambda x: x[1]` equals `def func(x): return x[1]`

In [7]:
tuples = [(1,'d'), (2, 'f'), (3, 'a'), (4, 'b'), (9, 'c')]

In [11]:
sorted(tuples, key = lambda x: x[1])

[(3, 'a'), (4, 'b'), (9, 'c'), (1, 'd'), (2, 'f')]

# Decorators

In [121]:
#decorator is used to modify existing function without rewriting it
def uppercase(func):
    def wrapper(text):
        """here goes wrong docs - wrapper's docs"""
        original_result = func(text)
        modified_result = original_result.upper()
        return modified_result
    return wrapper

@uppercase
def print_hw(text):
    """this is useful docs"""
    return text

print_hw('hello world')

'HELLO WORLD'

In [126]:
# there's a caveat:
print('DOCSTRINGS are wrong: "{}"\nand NAME is also wrong: "{}"'.format(print_hw.__doc__, print_hw.__name__))

DOCSTRINGS are wrong: "here goes wrong docs - wrapper's docs"
and NAME is also wrong: "wrapper"


In [130]:
#the same as
def upper_it(func):
    orig = func()
    modif = orig.upper()
    return modif

def greet():
    """return friendly greeting"""
    return 'Hello'

greets = upper_it(greet)
greets

'HELLO'

In [131]:
greet.__doc__

'return friendly greeting'

### Nesting decorators 

In [145]:
def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper

def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper

@emphasis
@strong      
def greet():
    return 'hello from HTML!'
greet()

'<em><strong>hello from HTML!</strong></em>'

In [141]:
def trace(func):
    def wrapper(*args, **kwargs):
        '''not so much useful docs'''
        print(f'TRACE: calling {func.__name__}() with {args}, {kwargs}')
        original_result = func(*args, **kwargs)
        print(f'TRACE: {func.__name__}() returned {original_result!r}')
        return original_result
    return wrapper
              

In [142]:
@trace
def say(name, line):
    '''VERY useful docs'''
    return f'{name}, {line}'

In [156]:
say('Jane', 'hi!')
print(say.__doc__, "----- > returns wrapper's docs")
print(say.__name__, "--------------------  > returns wrapper's name")

TRACE: calling say() with ('Jane', 'hi!'), {}
TRACE: say() returned 'Jane, hi!'
not so much useful docs ----- > returns wrapper's docs
wrapper --------------------  > returns wrapper's name


### functools decorator workaround for docstrings

In [157]:
from functools import wraps

def uppercase(func):
    @wraps(func)
    def wrapper(text):
        """wrapper docs"""
        original_result = func(text)
        modified_result = original_result.upper()
        return modified_result
    return wrapper

@uppercase
def print_hw(text):
    """this is useful docs"""
    return text

print_hw('hello world')

'HELLO WORLD'

In [158]:
print_hw.__doc__

'this is useful docs'

In [159]:
print_hw.__name__

'print_hw'

### shallow and deep copies

In [170]:
xs = [[1,2,3], [4,5,6], [7,8,9]]
ys = list(xs) # shallow copy, has child objects (lists whithin list)

In [171]:
id(xs)

1883577920

In [172]:
id(ys)

1882935824

In [176]:
xs[0][1]='X'

In [177]:
print(ys)

[[1, 'X', 3], [4, 5, 6], [7, 8, 9]]


In [185]:
s = 2112
y = 8
print(f'{s:>{y}}')

    2112


In [186]:
import copy
zs = copy.deepcopy(xs)

In [187]:
zs

[[1, 'X', 3], [4, 5, 6], [7, 8, 9]]

In [188]:
xs[0][1] = 'Y'

In [189]:
zs

[[1, 'X', 3], [4, 5, 6], [7, 8, 9]]

In [191]:
xs

[[1, 'Y', 3], [4, 5, 6], [7, 8, 9]]

In [199]:
# copying objects
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point({self.x!r}, {self.y!r})'
    
p = Point(1,2)
print(p)
s = copy.copy(p)
print(s)
print(s is p)

Point(1, 2)
Point(1, 2)
False


In [204]:
class Rectangle:
    def __init__(self, topleft, bottomright):
        self.topleft = topleft
        self.bottomright = bottomright
        
    def __repr__(self):
        return (f'Rectangle({self.topleft!r}, {self.bottomright!r})')
    
rect = Rectangle(Point(0,1), Point(5,6))
srect = copy.copy(rect)
print(rect, srect)
rect == srect

Rectangle(Point(0, 1), Point(5, 6)) Rectangle(Point(0, 1), Point(5, 6))


False

In [205]:
rect.topleft.x = 999
rect

Rectangle(Point(999, 1), Point(5, 6))

In [206]:
srect

Rectangle(Point(999, 1), Point(5, 6))

In [211]:
drect = copy.deepcopy(srect)
drect.topleft.x = 222
print(rect, srect, drect)

Rectangle(Point(999, 1), Point(5, 6)) Rectangle(Point(999, 1), Point(5, 6)) Rectangle(Point(222, 1), Point(5, 6))


In [1]:
class Pizza:
    def __init__(self, ingredients):
        self.ingredients = ingredients

    def __repr__(self):
        return f'Pizza({self.ingredients!r})'

    @classmethod
    def margherita(cls):
        return cls(['mozzarella', 'tomatoes'])

    @classmethod
    def prosciutto(cls):
        return cls(['mozzarella', 'tomatoes', 'ham'])

In [3]:
pm = Pizza.margherita()
print(pm)

Pizza(['mozzarella', 'tomatoes'])


In [25]:
import math
class Pizza:
    def __init__(self, radius, ingredients):
        self.ingredients = ingredients
        self.radius = radius

    def __repr__(self):
        return f'Pizza(radius: {self.radius!r}, ingredients: {self.ingredients!r})'
    
    def area(self):
        return self.circle_area(self.radius)
    
    @staticmethod
    def circle_area(r):
        return r**2*math.pi
    
    @classmethod
    def margherita(cls):
        return cls(['mozzarella', 'tomatoes'])

    @classmethod
    def prosciutto(cls):
        return cls(['mozzarella', 'tomatoes', 'ham'])
    
p = Pizza(4, ['mozzarella', 'tomatoes'])
print(p)

Pizza(radius: 4, ingredients: ['mozzarella', 'tomatoes'])


In [26]:
p

Pizza(radius: 4, ingredients: ['mozzarella', 'tomatoes'])

In [27]:
Pizza.circle_area(4)

50.26548245743669

In [28]:
p.area()

50.26548245743669